# EmergencyZIP AI

## Intro

As a Data Scientist in Healthcare with over two years of industry experience, I’ve learned that a person’s zip code (postal code) plays a crucial role in determining their health status. Where an individual lives can significantly impact their overall health, as local healthcare quality varies across regions. Unfortunately, many people are unaware of the healthcare facilities available in their area or may not know which one is best suited for their specific medical needs. To address this, I developed EmergencyZIP AI, a tool designed to help individuals identify the best medical facilities based on their symptoms or conditions, while considering their zip code as a critical factor in their health status.

## Use Case and Solution Approach

Individuals often seek clarity about the illness or medical condition they may be experiencing and want to know which nearby healthcare facilities can provide the best treatment. EmergencyZIP AI utilizes advanced AI to not only help individuals identify the potential illness or condition they’re facing but also guide them to the most appropriate medical facilities nearby based on zip code.

The system takes zip codes into account, recognizing the importance of location in determining healthcare accessibility and quality.

## Innovation/Novelty

Although the internet provides many resources for finding local healthcare facilities, searching through numerous results can be overwhelming. Moreover, it’s difficult for individuals to accurately diagnose their condition without medical expertise. EmergencyZIP AI solves this by offering personalized and tailored responses, focusing specifically on the individual’s symptoms and their location.

While human doctors are skilled at diagnosing, they may still be influenced by biases or errors. AI, on the other hand, minimizes these biases by considering the patient’s demographic information. However, I acknowledge that human doctors can recognize nuances that AI may miss, as AI is typically trained on generalized data. The goal of EmergencyZIP AI is to make medical information more accessible and less overwhelming for users, while providing individualized insights.

## Process

EmergencyZIP AI operates using Structured Output/JSON Mode/Controlled Generation, Few-Shot Prompting, and Grounding techniques. Here’s how it works:

1. JSON Mode: The AI processes the patient’s demographic information and current symptoms, converting it into a structured JSON format, similar to a simplified patient intake form.
2. Few-Shot Prompting: The AI generates a request based on the information provided by the JSON form using a few provided examples to maintain consistency and accuracy which is then included in a prompt for the AI asking for the most relevant data to help diagnose the condition. The AI also extracts zipcode from the JSON form using a few provided examples to maintain accuracy to be used to find nearby locations.
3. Grounding: Using the provided information, the AI offers the most accurate potential diagnosis for the individual’s symptoms. Then, it uses the zip code to find the nearest and most appropriate medical facilities for treatment. Then, the AI provides payment and insurance plan information for each recommended location.

### Importing Relevant Libraries and Technologies 

I set up Gemini API to implement my AI (the code also retrieves the API Key from Kaggle Secrets to retrieve the Google API that makes it possible for the AI to work) and imported the Pandas and Random libraries (and other unnamed libraries) to do my test run to check if the AI works as intended or not. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/patient-symptom-dataset/patient_symptom_data_with_demographics_imperial.csv


The cell below removes any irrelevant libraries that may conflict with the Google Generative AI capabilities and install the Google Gemini Generative AI package

In [2]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.9 MB/s eta 0:00:00


Install all relevant libraries for the Google Generative AI capabilities

In [3]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [4]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

Retrieve the Google API Key from Kaggle Secrets to make possible for the AI to work in this code.

In [5]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [6]:
client = genai.Client(api_key=GOOGLE_API_KEY)

## Generative AI Capabilities Used and Reasoning 

These are the 3 Generative AI Capabilities Used in this AI and why they are used in this AI: 
* **Structured Output/JSON Mode/Controlled Generation**: For JSON Mode, I want to somewhat mimic the forms that patients fill out when they are requesting care from a medical facility, but simplier and more abstract. For Structured Output and Controlled Generation, I want my responses to be as accurate and precise as possible where creativity may not be required but accuracy is definitely required to get the best responses as possible.
* **Few-Shot Prompting**: First of all, I want to create requests for the AI to know who it is providing information for, but I want the formatting of the requests to be as consistent as possible so that the information does not get muddled up which would make things more difficult for the AI to process and provide information for. Then, I also want to extract the zipcode from the JSON form, but sometimes the AI can hallucinate and return irrelevant or incorrect information, so I want the let the AI know how to return the zipcode correctly.
* **Grounding**: This is the central capability of the AI since the AI is responsible for finding all the relevant possible nearest locations to the patient based on postal code/zipcode. Since we are exploring so many zipcodes and it is possible that the AI does not know every zipcode or medical facilities per zipcode, we need to rely on external sources to get the best answers possible. However, since we don't have a document listing out every medical facility per zipcode or all zipcodes, we would need to rely on the internet to give us answers. 

### Generative AI Capability #1: Structured Output/JSON Mode/Controlled Generation

#### JSON Mode

I created a Python class, MedicalPatient, to format a simple and abstract patient form in JSON. Then, I developed a get_json_form function that takes an entry (in the test run that I did to test my AI, a row from a Pandas DataFrame) and structures it into the JSON format defined in the MedicalPatient class.

##### get_json_form function: 

**Description**: Function takes in an entry (in this case, a Pandas dataframe row) and structures the information provided into a formalized JSON form as structured in the MedicalPatients class. 

**Input**: ***patient_symptom_entry***: Dataframe Pandas row (for now) of all information needed to be processed as JSON form 

**Output**: ***json_form***: String of formalized JSON form

In [7]:
import typing_extensions as typing

class MedicalPatient(typing.TypedDict): 
    zipcode: int
    age: int
    gender: str
    race: str
    ethnicity: str
    height: float
    weight: float
    symptoms: list[str]

def get_json_form(patient_symptom_entry): 
    entry_str = patient_symptom_entry.to_string()
    json_response = client.models.generate_content(
        model='gemini-2.0-flash', 
        config=types.GenerateContentConfig(
            temperature=0.1, 
            response_mime_type="application/json", 
            response_schema=MedicalPatient
        ), 
        contents=entry_str
    )
    json_form = json_response.text
    return json_form

#### Structured Output/Controlled Generation

**Description**: I created the get_general_model_config function to return a Gemini-generated output based on the temperature, top_p, and maximum number of tokens to return inputs.

**Input**: 
* ***temperature***: double that reflects the degree of randomness in selecting tokens for output
* ***top_p***: double that is the maximum culmulative probability that the model can reach to select tokens as candidates
* ***max_output_tokens***: int that indicates the maximum number of tokens that should be provided and included in the input

**Output**: ***str_output***: String of Gemini-generated output based on input parameters provided

In [8]:
def get_general_model_config(temperature, top_p, max_output_tokens): 
    str_output = types.GenerateContentConfig(
        temperature=temperature, 
        top_p=top_p, 
        max_output_tokens=max_output_tokens
    )
    return str_output

### Generative AI Capability #2: Few-Shot Prompting

**Description**: I created the few_shot_func function to implement the few-shot prompting AI capability as code

**Input**: 
* ***model_input***: String that is the input (e.g. a JSON form) that the model takes in consideration of to provide an output response
* ***prompt***: String that is the input prompt to provide to the Gemini model to provide a response for; in few-shot prompting, the input prompt provides a few examples with their expected responses to the Gemini model so that the Gemini models knows what to produce for their output format-wise and what their output should look like
* ***config***: Python method that is configuration to be used for the Gemini model to produce output (*refer to get_general_model_config as example of Python method to use as possible parameter value*)

**Output**: ***str_output***: String of Gemini-generated output based on input parameters provided

In [9]:
def few_shot_func(model_input, prompt, config): 
    few_shot_response = client.models.generate_content(
        model='gemini-2.0-flash', 
        config=config,
        contents=[prompt, model_input]
    )
    str_response = few_shot_response.text
    return str_response

### Generative AI Capability #3: Grounding

**Description**: I created the grounding_func function to implement the grounding AI capability as code

**Input**: ***input_prompt***: String that is the input prompt to provide to the Gemini model to provide a response for

**Output**: ***str_response***: String that is the output provided by the Gemini model as response to the input prompt

In [10]:
def grounding_func(input_prompt): 
    config_with_search = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
    )
    response = client.models.generate_content(
        model='gemini-2.0-flash', 
        contents=input_prompt, 
        config=config_with_search,
    )
    rc = response.candidates[0]
    str_response = rc.content.parts[0].text
    return str_response

## EmergencyZIP AI Test Run

Kaggle Notebooks seem to not work or do well with Python input, so instead, we generated a dataset of patients with relevant demographic information and their symptoms through ChatGPT (so the dataset below is ChatGPT-generated), and then the code picks a random patient to test run the AI to make sure it actually works as intended. 

In [11]:
patient_symptom_df = pd.read_csv('/kaggle/input/patient-symptom-dataset/patient_symptom_data_with_demographics_imperial.csv')
patient_symptom_df

,PatientID,ZipCode,Age,Gender,Symptoms,Race,Ethnicity,Height_in,Weight_lb
0,P001,70112,32,Female,Joint pain;Runny nose;Loss of taste;Sore throat,Other,Hispanic or Latino,73.6,229.3
1,P002,90002,21,Female,Joint pain;Muscle ache;Dry cough,White,Hispanic or Latino,76.8,262.3
2,P003,20001,46,Male,Chest tightness;Sneezing;Fever;Breathing diffi...,Black or African American,Not Hispanic or Latino,67.3,187.4
3,P004,73301,45,Male,Fatigue;Abdominal pain,White,Not Hispanic or Latino,67.7,183.0
4,P005,96813,23,Non-binary,Rash;Sore throat;Loss of smell;Abdominal pain;...,Pacific Islander,Not Hispanic or Latino,74.8,211.6
...,...,...,...,...,...,...,...,...,...
95,P096,55414,80,Female,Blurred vision;Dry cough,Black or African American,Hispanic or Latino,66.5,264.6
96,P097,10001,88,Male,Joint pain;Palpitations,White,Not Hispanic or Latino,61.8,152.1
97,P098,55401,55,Non-binary,Diarrhea;Palpitations;Low-grade fever;Joint pain,Native American,Hispanic or Latino,68.5,163.1
98,P099,68102,83,Non-binary,Sneezing;Fatigue;Headache,Native American,Not Hispanic or Latino,78.7,251.3


### Patient Input and JSON Patient Form

I utilized the randint function from the Random library to select a random patient to process information and provide responses for. Then, I utilized JSON Mode through calling the get_json_form method to have the AI mimic filling out a patient form in a simple and abstract way, and to also have the patient/individual demographic information structured in a formalized way. 

In [12]:
patient_index = random.randint(0, 99)
patient_json_form = get_json_form(patient_symptom_df.iloc[patient_index])
print(patient_json_form)

{
  "zipcode": 90002,
  "age": 24,
  "gender": "Non-binary",
  "race": "Pacific Islander",
  "ethnicity": "Hispanic or Latino",
  "height": 63.4,
  "weight": 127.9,
  "symptoms": [
    "Muscle ache",
    "Loss of smell",
    "Sneezing",
    "Shortness of breath"
  ]
}


### AI Request

In this first round of few-shot prompting, I utilized few-shot prompting to create a formalized request for the AI model to know what type of patient or individual that the model will be retrieving information for. Few-shot prompting is also used to ensure accuracy and consistency of the format of the request (the model is always clear on what information to extract from the patient and how to process that information into a formalized request). We also used structured output/controlled generation to ensure that the model is consistent in generating the output response as possible. 

In [13]:
model_config = get_general_model_config(0.1, 1, 500)

few_shot_prompt = """Parse JSON into a string request and only return output: 

EXAMPLE: 
{
"zipcode": 78664, 
"age": 30, 
"gender": "Female", 
"race": "White", 
"ethnicity": "Not Hispanic or Latino", 
"height": 64.6, 
"weight": 130.5, 
"symptoms": ["Cough", "Fever", "Sore throat"]
}
"30-year old Not Hispanic or Latino White female patient with height of 64.6 inches and weight of 130.5 pounds from zipcode 78664 currently experiencing symptoms of cough, fever, and sore throat."

EXAMPLE: 
{
"zipcode": 77055, 
"age": 25, 
"gender": "Non-binary", 
"race": "Asian", 
"ethnicity": "Not Hispanic or Latino", 
"height": 60.5, 
"weight": 100.2, 
"symptoms": ["Joint pain", "Loss of smell"]
}
"25-year old Not Hispanic or Latino Asian non-binary patient with height of 60.5 inches and weight of 100.2 pounds from zipcode 77055 currently experiencing symptoms of joint pain and loss of smell."

EXAMPLE: 
{
"zipcode": 78681, 
"age": 15, 
"gender" "Male", 
"race": "Black", 
"ethnicity": "Hispanic or Latino", 
"height": 74.6, 
"weight": 180.5, 
"symptoms": ["Fatigue", "Loss of Vision", "Headache", "Joint pain"]
}
"15-year old Hispanic or Latino Black male patient with height of 74.6 inches and weight of 180.5 pounds from zipcode 78681 currently experiencing symptoms of fatigue, loss of vision, headache, and joint pain."
"""

patient_medical_request = few_shot_func(patient_json_form, few_shot_prompt, model_config)
print(patient_medical_request)

"24-year old Hispanic or Latino Pacific Islander non-binary patient with height of 63.4 inches and weight of 127.9 pounds from zipcode 90002 currently experiencing symptoms of muscle ache, loss of smell, sneezing, and shortness of breath."



### Potential Diagnosis

In this first round of grounding, I utilized grounding to provide a potential but precise diagnosis based on the symptoms and other relevant demographic information, especially zipcode, provided. I am not only utilizing the data that the AI has already been provided to conduct the diagnosis, but also other sources, like the internet as well. 

In [14]:
diagnosis_prompt = patient_medical_request + "What disease are they most likely to experience now? Only return the singular most likely disease, please."
diagnosis = grounding_func(diagnosis_prompt)
Markdown(diagnosis)

Based on the symptoms of muscle ache, loss of smell, sneezing, and shortness of breath, the most likely disease they are experiencing is COVID-19.


*I had to run the grounding process again since after the notebook restarts, the response can be hit-or-miss in terms of accuracy or helpfulness*

In [15]:
diagnosis_prompt = patient_medical_request + "What disease are they most likely to experience now? Only return the singular most likely disease, please."
diagnosis = grounding_func(diagnosis_prompt)
Markdown(diagnosis)

To determine the most likely disease, I need to consider the patient's symptoms, age, ethnicity, and other provided information. Given the symptoms of muscle ache, loss of smell, sneezing, and shortness of breath, I'll research possible conditions presenting with these symptoms, especially considering current common illnesses.



### Patient Zipcode 

In this second round of few-shot prompting, I utilized few-shot prompting to extract the zipcode of the patient from the JSON form (since at this point, we are using the JSON form to get our data information). Few-shot prompting is utilized to ensure accuracy of the output based on the current format of the JSON form (in this case, a simplified and abstract version of a patient form). 

In [16]:
model_config_two = get_general_model_config(0, 1, 50)

few_shot_prompt_two = """Parse JSON to return only the Zipcode: 

EXAMPLE: 
{
"zipcode": 78664, 
"age": 30, 
"gender": "Female", 
"race": "White", 
"ethnicity": "Not Hispanic or Latino", 
"height": 64.6, 
"weight": 130.5, 
"symptoms": ["Cough", "Fever", "Sore throat"]
}
78664

EXAMPLE: 
{
"zipcode": 77055, 
"age": 25, 
"gender": "Non-binary", 
"race": "Asian", 
"ethnicity": "Not Hispanic or Latino", 
"height": 60.5, 
"weight": 100.2, 
"symptoms": ["Joint pain", "Loss of smell"]
}
77055

EXAMPLE: 
{
"zipcode": 78681, 
"age": 15, 
"gender" "Male", 
"race": "Black", 
"ethnicity": "Hispanic or Latino", 
"height": 74.6, 
"weight": 180.5, 
"symptoms": ["Fatigue", "Loss of Vision", "Headache", "Joint pain"]
}
78681
"""

patient_zipcode = few_shot_func(patient_json_form, few_shot_prompt_two, model_config_two)
print(patient_zipcode)

90002



### Nearest Medical Facilities

In this second round of grounding, I utilized grounding to provide the nearest appropriate medical facilities for the individual to go to for treatment based on zipcode. We are not only utilizing the data that the AI has already been provided to find the nearest location, but also other sources, like the internet as well. 

*Note: Would not recommend to run the cell below more than once to get helpful and accurate information*

In [17]:
recommendations_prompt = diagnosis + 'Can you get me the nearest best-rated medical facilities with their addresses that can treat the disease for zip code ' + str(patient_zipcode) + '?'
recommendations = grounding_func(recommendations_prompt)
Markdown(recommendations)

Based on the symptoms you described—muscle ache, loss of smell, sneezing, and shortness of breath—it's important to consider conditions like COVID-19, the flu, or a common cold, as these share overlapping symptoms. Loss of smell is particularly associated with COVID-19, but it can also occur with a cold or sinus infection. Shortness of breath is more indicative of COVID-19 or the flu, but it's important to get a diagnosis from a medical professional.

Given your location (zip code 90002), here are some of the nearest medical facilities with potentially good ratings:

*   **Watts Healthcare Corporation**

    *   **Watts Health Center:** 10300 Compton Ave Los Angeles, CA 90002. Phone: (323) 564-4331
    *   **Watts Healthcare – Jordan Wellness Center:** 10110 Juniper St Los Angeles, CA 90002. Phone: (323) 564-4331
*   **Family & Community Medical Clinic:** 9901 S. Compton Avenue Los Angeles CA 90002. Phone: (323) 835-

It's important to note that "best-rated" can be subjective and may depend on the specific criteria used for evaluation.


### Payment and Insurance Plan Information

In this third round of grounding, I utilized grounding to provide payment and insurance plan information for each of the location recommended for treatment. We are not only utilizing the data that the AI has already been provided to retrieve the information, but also other sources, like the internet as well. 

*Note: Would not recommend to run the cell below more than once to get helpful and accurate information*

In [18]:
payments_prompt = "For each of the locations recommended to me: " + recommendations + " what types of payments and insurance plans are accepted for patients coming in for treatment?"
payments = grounding_func(payments_prompt)
Markdown(payments)

It's crucial to confirm the most up-to-date information directly with each facility, as insurance plans and payment policies can change. Here's a breakdown based on the available information:

**1. Watts Healthcare Corporation & Watts Healthcare - Jordan Wellness Center (Phone: (323) 564-4331):**

*   **Insurance:** Watts Healthcare accepts "most insurance plans including Medi-Cal Managed Care options, California Health and Wellness and Anthem Blue Cross Partnership Plans, Medicare". They also have contracts with various healthcare coverage programs.
*   **Specific Plans Mentioned:**
    *   Anthem Blue Cross
    *   Blue Shield of CA Promise Health Plan
    *   Health Net
    *   LA Care
    *   Molina
    *   Medicare Advantage HMO plans like Alignment, Brand New Day, Central Health Plan
*   **Payment Options:**
    *   Private Pay
    *   Sliding Fee Discount Program: Watts Healthcare Corporation is committed to providing a sliding fee discount to persons who have healthcare needs and are uninsured, underinsured, ineligible for a government program, or otherwise unable to pay.
    *   They can assist with applying for Medicaid, Medicare, or insurance plans through California's marketplace.
*   **Other Important Information:**
    *   They offer a sliding fee scale for those without insurance, but you must bring proof of residence (light or water utility bill), ID, and two recent pay stubs.

**2. Family & Community Medical Clinic (Phone: (323) 835-):**

*   I am unable to find specific information about the Family & Community Medical Clinic's accepted insurance plans and payment options. It is best to contact them directly.



## Limitations and Possible Next Steps

While EmergencyZIP AI functions as intended, it is a basic implementation due to time constraints. Some advanced features were not included due to limitations in the development process (e.g., Python input errors). Additionally, grounding-based responses can sometimes be inaccurate, so the AI should not be considered a replacement for professional medical advice. It provides potential diagnoses and recommendations that should be treated as informational rather than authoritative.

Future improvements include:

* Transitioning the AI into an AI Agent using LangGraph.
* Implementing an interactive feature where users input their information, and the AI processes and responds with relevant recommendations.
* Developing a UX/UI interface for a more user-friendly experience.
* Expanding the JSON form to capture additional information, such as address, health history, and income.

## Conclusion and Potential Impact 
I created EmergencyZIP AI to provide individuals with a more individualized and accessible way to understand their symptoms, find appropriate treatment options, and consider the crucial role of zip codes in determining their health. While the current version is basic due to time constraints, it functions as intended, helping users make informed decisions about their healthcare.

The goal of this AI is to empower individuals to take charge of their health by providing them with easy access to healthcare resources. Ultimately, I hope EmergencyZIP AI will contribute to improving health outcomes in various regions, one zip code at a time. By helping individuals become more health-conscious and informed, this tool has the potential to make a significant impact on healthcare access and overall well-being.